# Import Libraries

In [ ]:
import copy
import utils
import torch
import constants
import numpy as np
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.distributions import Normal, Categorical

from env import Env
from agent import Agent
from torchsummary import summary

# Initialise Environment

In [ ]:
#initialise environment
min_x, max_x =  -0.110 - 0.175,   -0.110 + 0.175
min_y, max_y =   0.510 - 0.175,    0.510 + 0.175
min_z, max_z =               0,              0.4 

workspace_lim = np.asarray([[min_x, max_x], 
                            [min_y, max_y],
                            [min_z, max_z]])

print(f"workspace space: \n{workspace_lim}")

obj_dir = 'objects/blocks/'
N_obj   = 10

env = Env(obj_dir, N_obj, workspace_lim)

# Initialise Agent

In [ ]:
agent = Agent(env, max_memory_size = 100)

# Gather Demonstration Experience

In [ ]:
# agent.gather_guidance_experience(max_episode = 15)

# Get Train and Test Loader

In [ ]:
agent.buffer_replay.load_buffer()

grasp_exp = agent.buffer_replay.get_experience_by_action_type(constants.GRASP)
push_exp  = agent.buffer_replay.get_experience_by_action_type(constants.PUSH)
hld_exp   = agent.buffer_replay.get_experience_by_home_type()

print(f"N_grasp_exp: {len(grasp_exp[0])}")
print(f"N_push_exp: {len(push_exp[0])}")
print(f"N_HLD_exp: {len(hld_exp[0])}")

hld_train_loader, hld_test_loader     = agent.get_train_test_dataloader_hld_net(hld_exp)
grasp_train_loader, grasp_test_loader = agent.get_train_test_dataloader(grasp_exp, is_grasp = True)
push_train_loader, push_test_loader   = agent.get_train_test_dataloader( push_exp, is_grasp = False)

for a,b,c in push_test_loader:
    print("hi")

# HLD-net clone

In [ ]:
print(hld_exp[0].shape, hld_exp[1].shape, hld_exp[2].shape)
index = 11
print(hld_exp[2][index])
plt.imshow(hld_exp[1][index])

In [ ]:
agent.behaviour_cloning_hld(hld_train_loader, agent.hld_net, num_epochs = 10)
agent.behaviour_cloning_eval_hld(hld_test_loader, agent.hld_net)

# Push Clone

In [ ]:
agent.behaviour_cloning(push_train_loader, agent.push_critic1, agent.push_critic2, agent.push_actor, num_epochs = 10, is_grasp = False)
agent.behaviour_cloning_eval(push_test_loader, agent.push_critic1, agent.push_critic2, agent.push_actor, is_grasp = False)

# Grasp Clone

In [ ]:
agent.behaviour_cloning(grasp_train_loader, agent.grasp_critic1, agent.grasp_critic2, agent.grasp_actor, num_epochs = 10, is_grasp = True)
agent.behaviour_cloning_eval(grasp_test_loader, agent.grasp_critic1, agent.grasp_critic2, agent.grasp_actor, is_grasp = True)